<center><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks">unSkript Runbooks</h1>
<div class="alert alert-block alert-success">
<h3 id="Objective"><strong>Objective</strong></h3>
<strong>To&nbsp;Launch AWS EC2 from AMI using unSkript actions.</strong></div>
</center><center>
<h2 id="Launch-AWS-EC2-from-AMI">Launch AWS EC2 from AMI</h2>
</center>
<h1 id="Steps-Overview">Steps Overview</h1>
<ol>
<li>Launch AWS EC2 Instance From an AMI</li>
<li>Get AWS Instances Details</li>
</ol>

<h3 id="Launch-AWS-EC2-Instance-From-an-AMI">Launch AWS EC2 Instance From an AMI</h3>
<p>Here we will use unSkript <strong>Launch AWS EC2 instance from an AMI</strong> action. This action is used to launch an EC2 instance from AMI.</p>
<blockquote>
<p><strong>Input parameters:</strong> <code>ami_id, region</code></p>
</blockquote>
<blockquote>
<p><strong>Output variable:</strong> <code>launch_instance</code></p>
</blockquote>

In [9]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List
import pprint


from beartype import beartype
@beartype
def aws_launch_instance_from_ami_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_launch_instance_from_ami(handle, ami_id: str, region: str) -> List:
    """aws_launch_instance_from_ami Launch instances from a particular image.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type ami_id: string
        :param ami_id: AMI Id Information required to launch an instance.

        :type region: string
        :param region: Region to filter instances.

        :rtype: Dict with launched instances info.
    """
    ec2Client = handle.client('ec2', region_name=region)

    res = ec2Client.run_instances(ImageId=ami_id, MinCount=1, MaxCount=1)

    return res['Instances']


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "ami_id": "iter_item",
    "region": "region"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "ami_id",
    "iter_parameter": "ami_id"
    }''')
task.configure(outputName="launch_instance")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_launch_instance_from_ami, lego_printer=aws_launch_instance_from_ami_printer, hdl=hdl, args=args)

<h3 id="Modify-Output">Modify Output</h3>
<p>In this action, we sort the output from step-1 and get the instance ids.</p>

In [15]:
instance_ids = []
if launch_instance:
    for k, v in launch_instance.items():
        for i in v:
            instance_ids.append(i["InstanceId"])

<h3 id="Get-AWS-Instance-Details">Get AWS Instance Details</h3>
<p>Here we will use unSkript <strong>Get AWS Instance Details</strong> action to get the details of the instances. This action is used to get details of instances that we received in step 1.</p>
<blockquote>
<p><strong>Input parameters:</strong> <code>instance_id, region</code></p>
</blockquote>
<blockquote>
<p><strong>Output variable:</strong> <code>instance_details</code></p>
</blockquote>

In [16]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Dict
import pprint


from beartype import beartype
@beartype
def aws_get_instances_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_get_instance_details(handle, instance_id: str, region: str) -> Dict:
    """aws_get_instance_details Returns instance details.

        :type handle: object
        :param handle: Object returned by the task.validate(...) method.

        :type instance_ids: list
        :param instance_ids: List of instance ids.

        :type region: string
        :param region: Region for instance.

        :rtype: Dict with the instance details.
    """

    ec2client = handle.client('ec2', region_name=region)
    instances = []
    response = ec2client.describe_instances(
        Filters=[{"Name": "instance-id", "Values": [instance_id]}])
    for reservation in response["Reservations"]:
        for instance in reservation["Instances"]:
            instances.append(instance)

    return instances[0]


def unskript_default_printer(output):
    if isinstance(output, (list, tuple)):
        for item in output:
            print(f'item: {item}')
    elif isinstance(output, dict):
        for item in output.items():
            print(f'item: {item}')
    else:
        print(f'Output for {task.name}')
        print(output)

task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "region": "region",
    "instance_id": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "instance_ids",
    "iter_parameter": "instance_id"
    }''')
task.configure(outputName="instance_details")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_instance_details, lego_printer=unskript_default_printer, hdl=hdl, args=args)

<h3 id="Instance-Details">Instance Details</h3>
<p>In this action, we sort the output from step-2 and present the details of the instance in the good table.</p>

In [17]:
import pprint
from pydantic import BaseModel, Field
from typing import Dict
from tabulate import tabulate
task.configure(printOutput=True)

output = instance_details
instance_list = instance_ids


def Instance_Details(output, instance_list: list):
    data1 = []
    Header = ""
    for instance_id in instance_list:
        if instance_id in output.keys():
            output1 = output[instance_id]
            if isinstance(output1, (list, tuple)):
                for item in output1:
                    print(f'item: {item}')
            elif isinstance(output1, dict):
                for key, value in output1.items():
                    if isinstance(value, (list)):
                        pass
                    else:
                        if key == "InstanceId":
                            Header = value
                        data1.append([key,value])
                print(f'\n\033[1m Table for Instance ID : {Header} \033[0;0m')
                print(tabulate(data1))
            else:
                print(f'Output for {task.name}')
                print(output1)


Instance_Details(output, instance_list)

### Conclusion
<p>In this Runbook, we demonstrated the use of unSkript's AWS legos to perform AWS actions and this runbook launched EC2 instances from AMI and show the details of the instance. To view the full platform capabilities of unSkript please visit <a href="https://us.app.unskript.io" target="_blank" rel="noopener">https://us.app.unskript.io</a></p>